### 8.일단 어느정도 전처리가 되었습니다. 랜덤 포레스트와 GBM 머신러닝 모델로 학습을 진행하겠습니다.
- 학습을 진행 하다가 성능이 좋지 않으면 데이터를 더 확인해서 전처리를 진행하겠습니다.

In [1]:
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from category_encoders import OrdinalEncoder

In [18]:
url = '/Users/sunghoheo/section6/choco 회사 예측 프로젝트/AI_05_허성호(데이터파일)/preparedata.csv'

df = pd.read_csv(url)
df.columns

Index(['company', 'specific_bean_origin_or_bar_name', 'review_date',
       'cocoa_percent', 'company_location', 'rating', 'Bean_Type',
       'country_of_bean_origin', 'trinitario_bean', 'criollo_bean',
       'blend_bean', 'forastero_bean', 'new_rating'],
      dtype='object')

In [3]:
#이제 훈련데이터와 검증데이터로 나누겠습니다.
target = ['rating','new_rating']
x = df.drop(target, axis=1).copy()
y = df['rating']
X_train, X_test, y_train, y_test = train_test_split(x, y, shuffle=True,random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle=True,random_state=0)

In [4]:
#파이프라인 생성
gb_pipe = make_pipeline(
          OrdinalEncoder(),
          SimpleImputer(),#bean_type과 다른 코코아콩 종류에 결측치가 많아서 사용.
          GradientBoostingClassifier(n_estimators=100,
                                    random_state=2,
                                    max_depth=7)
)
gb_pipe.fit(X_train,y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['company',
                                      'specific_bean_origin_or_bar_name',
                                      'company_location', 'Bean_Type',
                                      'country_of_bean_origin',
                                      'trinitario_bean', 'criollo_bean',
                                      'blend_bean', 'forastero_bean'],
                                mapping=[{'col': 'company',
                                          'data_type': dtype('O'),
                                          'mapping': Ambrosia               1
Malai                  2
Kah Kow                3
Christopher Elbow      4
The Barn               5
                    ... 
By Cacao             469
Feitoria Cac...
forastero (arriba) ass      5
forastero (arriba)          6
forastero (arriba) asss     7
forastero, trinitario       8
criollo, forastero          9
blend-forastero,criollo    10
forastero(ar

In [5]:
pred = gb_pipe.predict(X_val)
test_pred= gb_pipe.predict(X_test)
val_acc = accuracy_score(y_val,pred)
test_acc = accuracy_score(y_test,test_pred)
print('GBM 검증 정확도:',val_acc)
print('GBM 테스트 정확도:',test_acc)

GBM 검증 정확도: 0.7427055702917772
GBM 테스트 정확도: 0.7422885572139304


In [6]:
random = make_pipeline(
         OrdinalEncoder(),
         SimpleImputer(),
         RandomForestClassifier(n_estimators=100,
                                    random_state=2,
                                    max_depth=7))
random.fit(X_train,y_train)

#검증
random_pred = random.predict(X_val)
random_acc = accuracy_score(y_val,random_pred)
#테스트
random_test_pred = random.predict(X_test)
random_test_acc = accuracy_score(y_test,random_test_pred)

print('랜덤 포레스트 검증 정확도:',random_acc)
print('랜덤 포레스트 테스트 정확도:',random_test_acc)

랜덤 포레스트 검증 정확도: 0.7015915119363395
랜덤 포레스트 테스트 정확도: 0.708457711442786


In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, shuffle=True,random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle=True,random_state=0)

In [8]:
#학습전에 ordinal,simpimputer를 통해 데이터를 변환하겠습니다.
encoder = OrdinalEncoder()
sim = SimpleImputer() 
#encoder
X_train_encoder = encoder.fit_transform(X_train)
X_val_encoder = encoder.transform(X_val)
X_test_encoder = encoder.transform(X_test)

#imputer
X_train_imputer = sim.fit_transform(X_train_encoder)
X_val_imputer = sim.transform(X_val_encoder)
X_test_imputer = sim.transform(X_test_encoder)

### 9.간단하게 하이퍼 파라미터 튜닝을 하지않고 모델링을 해보았는데 GBM 과 랜덤포레스트의 테스트데이터를 넣었을때 성능이 생각보다 많이 차이가 납니다.
* 이번에는 하이퍼 파라미터 튜닝을 거쳐서 학습을 진행하겠습니다.

In [9]:
#GBM모델 먼저 진행하겠습니다.
gbm_hy = {'learning_rate':[0.01,0.05,0.1], 'n_estimators':[100,250,500]}

gbm_tuning = GridSearchCV(estimator=GradientBoostingClassifier(max_depth=10, min_samples_split=2, min_samples_leaf=1, random_state=10), 
            param_grid = gbm_hy, scoring='accuracy',n_jobs=4)

gbm_tuning.fit(X_train_imputer,y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(estimator=GradientBoostingClassifier(max_depth=10,
                                                  random_state=10),
             n_jobs=4,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'n_estimators': [100, 250, 500]},
             scoring='accuracy')

In [10]:
#검증
val_pred = gbm_tuning.predict(X_val_imputer)
val_acc2 = accuracy_score(y_val,val_pred)
#테스트
test_pred= gbm_tuning.predict(X_test_imputer)
test_acc2 = accuracy_score(y_test,test_pred)
print('GBM 검증 정확도:',val_acc2)
print('GBM 테스트 정확도:',test_acc2)#유의미하게 많이 오른듯합니다. 랜덤포레스트에도 적용하겠습니다.

GBM 검증 정확도: 0.7745358090185677
GBM 테스트 정확도: 0.7830845771144279


In [11]:
#랜덤포레스트 GridSearchCV를 통한 하이퍼파라미터 탐색.
params = { 'n_estimators' : [5,10,15,20,50,100],
           'max_depth' : [20,30,40],
           'min_samples_leaf' : [6, 8, 10],
            'max_leaf_nodes': [10,20,30]
            }


# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
random_clf = RandomForestClassifier(random_state = 2)
random_grid = GridSearchCV(random_clf, param_grid = params, cv = 5, n_jobs = -1)
random_grid.fit(X_train_imputer, y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2), n_jobs=-1,
             param_grid={'max_depth': [20, 30, 40],
                         'max_leaf_nodes': [10, 20, 30],
                         'min_samples_leaf': [6, 8, 10],
                         'n_estimators': [5, 10, 15, 20, 50, 100]})

In [12]:
print('최적 하이퍼 파라미터: ', random_grid.best_params_)
print('최고 예측: {:.4f}'.format(random_grid.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 20, 'max_leaf_nodes': 30, 'min_samples_leaf': 6, 'n_estimators': 50}
최고 예측: 0.7327


In [13]:
#위에서 나온 결과대로 다시학습
random_clf1 = RandomForestClassifier(n_estimators = 50, 
                                max_depth = 20,
                                max_leaf_nodes = 30, 
                                min_samples_split = 6,
                                random_state = 2,
                                n_jobs = -1)
random_clf1.fit(X_train_imputer, y_train)
random_test_pred = random_clf1.predict(X_test_imputer)
print('예측 정확도: {:.4f}'.format(accuracy_score(y_test,random_test_pred)))

예측 정확도: 0.7095


### 랜덤포레스트의 경우 성능이 오르긴 올랐으나 그다지 유의미한 결과만큼 오르지는 않았습니다.
### GBM의 성능이 좋은 성능을 보여주기때문에 최종모델은 GBM으로 하겠습니다.

In [16]:
joblib.dump(gbm_tuning, './gbm_model.pkl')

['Chocomodel.joblib']